# Solution to Murder Mystery

Details of the problem: [Murder Mystery](https://mystery.knightlab.com/)

In [ ]:
import sqlite3
import pandas as pd
pd.set_option('display.max_colwidth', None)
conn = sqlite3.connect("./sql-murder-mystery.db")
cursor = conn.cursor()

# Take a look at the database

Retrieve list of databases

In [ ]:
cursor.execute("PRAGMA database_list").fetchall()

View all tables in the database

In [ ]:
cursor.execute("SELECT name \
               FROM sqlite_master \
               WHERE type='table';")
tables = cursor.fetchall()
print("Tables in the database: ")
for table in tables:
  print(table)


In [ ]:
table_name = 'crime_scene_report'
sql_query = f"SELECT sql \
                FROM sqlite_master \
                where name = '{table_name}'"
pd.read_sql_query(sql_query, conn)

# Solve the murder mystery step by step

Start with the hint: A crime has taken place and the detective needs your help. The detective gave you the crime scene report, but you somehow lost it. You vaguely remember that the crime was a ​murder​ that occurred sometime on ​Jan.15, 2018​ and that it took place in ​SQL City​.

We’ll focus on two key details:



*     The crime type is murder
*    The date is January 15, 2018





Let’s start by querying the crime_scene_report table to see what we can find based on that information.

In [ ]:
query = '''
SELECT *
FROM crime_scene_report
WHERE type='murder' AND city='SQL City' AND date="20180115"
'''
pd.read_sql_query(query, conn)

,date,type,description,city
0,20180115,murder,"Security footage shows that there were 2 witnesses. The first witness lives at the last house on ""Northwestern Dr"". The second witness, named Annabel, lives somewhere on ""Franklin Ave"".",SQL City


There are two witnesses—each with a name and an address. We know that the **person** table contains personal details, and the **address_street_name** field holds information about their street addresses. In addition, we can check the **interview** table to see what each witness had to say.

Let’s explore what we can find using these tables.

In [ ]:
query='''SELECT *
        FROM person
        JOIN interview ON interview.person_id = person.id
        WHERE
          address_street_name LIKE '%Northwestern Dr%' AND address_number = (
            SELECT MAX(address_number) FROM person WHERE address_street_name ='Northwestern Dr')
          OR (address_street_name LIKE '%Franklin Ave%' AND name like '%Annabel%' )
'''
# pd.read_sql_query(query, conn)

In [ ]:
query = '''
  WITH witnessA AS (
          SELECT *
          FROM person
          WHERE address_street_name LIKE '%Northwestern Dr%'
          ORDER BY address_number DESC
          LIMIT 1
  ), witnessB AS (
    SELECT *
    FROM person
    WHERE address_street_name LIKE '%Franklin Ave%' AND name like '%Annabel%'
  ), witness AS (
    SELECT *
    FROM witnessA
    UNION
    SELECT *
    FROM witnessB
  )

  SELECT witness.name,interview.transcript
  FROM witness
  JOIN interview ON interview.person_id = witness.id
'''

pd.read_sql_query(query, conn)

,name,transcript
0,Morty Schapiro,"I heard a gunshot and then saw a man run out. He had a ""Get Fit Now Gym"" bag. The membership number on the bag started with ""48Z"". Only gold members have those bags. The man got into a car with a plate that included ""H42W""."
1,Annabel Miller,"I saw the murder happen, and I recognized the killer from my gym when I was working out last week on January the 9th."


Now we’ve got another clue from the witness interviews. The suspect worked out at the gym, carried a gym bag with an ID starting with "48Z", a gold member, drove a car with a license plate containing "H42W", and was at the gym on January 9th.

In [ ]:
query = '''SELECT person.id, person.name
          FROM get_fit_now_member  AS gf
          JOIN person ON person.id=gf.person_id
          JOIN drivers_license ON drivers_license.id = person.license_id
          JOIN get_fit_now_check_in AS check_in ON check_in.membership_id = gf.id
          WHERE membership_status="gold" AND gf.id like '48Z%' AND plate_number LIKE '%H42W%' AND check_in_date='20180109'
'''
# pd.read_sql_query(query, conn)


In [ ]:

query = '''SELECT person.id, person.name, interview.transcript
          FROM get_fit_now_member  AS gf
          JOIN person ON person.id=gf.person_id
          JOIN drivers_license ON drivers_license.id = person.license_id
          JOIN get_fit_now_check_in AS check_in ON check_in.membership_id = gf.id
          JOIN interview ON interview.person_id = person.id
          WHERE membership_status="gold" AND gf.id like '48Z%' AND plate_number LIKE '%H42W%' AND check_in_date='20180109'
'''
pd.read_sql_query(query, conn)

,id,name,transcript
0,67318,Jeremy Bowers,"I was hired by a woman with a lot of money. I don't know her name but I know she's around 5'5"" (65"") or 5'7"" (67""). She has red hair and she drives a Tesla Model S. I know that she attended the SQL Symphony Concert 3 times in December 2017.\n"


The suspect was hired by another woman. Let's find out who she is
We got a hint that this woman height is between 65 and 67 inch, drives Tesla Model S and attended Symphony Concert 3 times in December 17

In [ ]:
query = '''
  SELECT person.id AS person_id, person.name, drivers_license.*
  FROM drivers_license
  JOIN person ON person.license_id=drivers_license.id
  WHERE hair_color='red' AND car_make='Tesla' AND car_model='Model S' AND height >=65 AND height<=67
'''
# pd.read_sql_query(query, conn)

In [ ]:
query = '''
  SELECT  person.id AS person_id, person.name, count(event_name) AS event_num
  FROM drivers_license
  JOIN person ON person.license_id=drivers_license.id
  JOIN facebook_event_checkin f ON f.person_id = person.id
  WHERE hair_color='red' AND
  car_make='Tesla' AND car_model='Model S' AND height >=65 AND height<=67 AND event_name = 'SQL Symphony Concert' AND date like '201712%'
  GROUP BY person.id
'''
pd.read_sql_query(query, conn)

,person_id,name,event_num
0,99716,Miranda Priestly,3


Hooray, we found the brain behind the murder is Miranda Priestly



In [ ]:
cursor.close()